### Importing the Libraries

In [ ]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import LSTM
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

### Loading the Dataset, applying Padding and Splitting the data

In [ ]:
# load the dataset but only keep the top n words, zero the rest
top_words = 8000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

# pad dataset to a maximum review length in words
max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

X = numpy.concatenate((X_train, X_test), axis=0)
y = numpy.concatenate((y_train, y_test), axis=0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42) # split into train and test sets.

### Defining the Model Architecture

In [ ]:
model = Sequential()
model.add(Embedding(top_words, 100, input_length=max_words))
model.add(LSTM(100,return_sequences=True))
model.add(Conv1D(64, 3, padding='same',activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dropout(0.8))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.8))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

### Training the Model


In [ ]:
# Fit the model
print('model compiled')
model.fit(X_train, y_train, validation_split=0.2, epochs=3, batch_size=128, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

model compiled
Instructions for updating:
Use tf.cast instead.
Train on 28000 samples, validate on 7000 samples
Epoch 1/3
 - 489s - loss: 0.4656 - acc: 0.7486 - val_loss: 0.2896 - val_acc: 0.8794
Epoch 2/3
 - 487s - loss: 0.2265 - acc: 0.9134 - val_loss: 0.3021 - val_acc: 0.8777
Epoch 3/3
 - 487s - loss: 0.1712 - acc: 0.9352 - val_loss: 0.2976 - val_acc: 0.8824
Accuracy: 88.49%


### Evaluating the Model

In [ ]:
test_size=len(X_test)
y_pred=model1.predict_proba(X_test)
for i in range(test_size):
    if y_pred[i]>0.4:
        y_pred[i]=1
    else:
        y_pred[i]=0

In [ ]:
cm=confusion_matrix(y_test,y_pred)
print(cm)

[[6750  723]
 [ 963 6564]]


In [ ]:
tp=cm[0][0]
fn=cm[0][1]
fp=cm[1][0]
tn=cm[1][1]

errorrate=((fp+fn)/(tp+fn+fp+tn))
accuracy=((tp+tn)/(tp+fn+fp+tn))
recall=(tp/(tp+fn))
specificity=(tn/(tn+fp))
precision=(tp/(tp+fp))
# fpr=1-specificity
f1_score=(2*precision*recall)/(precision+recall)

In [ ]:
print("Error Rate")
print(errorrate * 100)
print("Accuracy")
print(accuracy * 100)
print("Recall")
print(recall * 100)
print("Specificity")
print(specificity * 100)
print("Precision")
print(precision * 100)
'''
print("fpr")
print(fpr)
'''
print("F1 Score")
print(f1_score * 100)

ERRORRATE
0.1124
accuracy
0.8876
sensitivity
0.9032517061421116
specificity
0.8720605819051415
precision
0.8751458576429405
fpr
0.12793941809485854
f1_score
0.8889766890557093


### Saving the Model

In [ ]:
model.save("lstm_cnnmodel.h5")